In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to review the latest conversation first and output either '주문 변경' or '주문 취소'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
chain = prompt | model

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 1번째 경우

In [7]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 얼마인가요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run f1827218-99f4-4eec-8e31-e4cc548a8cd2 not found for run 5b044060-7af6-46c5-98d9-2de5d22b4e91. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 172, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-406e6eea-496f-4900-8c35-985c23c42006-0')

In [8]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 취소하고 싶은데요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 54f4ee2c-ba95-4914-a01e-5d6f1c0267ba not found for run cda60b06-feee-4bfc-ad92-de418c778fec. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 202, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a4f463dc-cfeb-4bb6-8920-fa06e6fbe777-0')

In [9]:
response = chain_with_memory.invoke(
    {"input": "product_nae: 떡케익 quantity: 5 price: 15,000", 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 04262e59-e424-4e42-a255-f7846e5574c5 not found for run 84e23f8b-515d-4cf9-9362-c74c587300e4. Treating as a root run.


AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 235, 'total_tokens': 240}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-244bbb49-d18a-4bea-8c01-c1f3eee741a6-0')

# 2번째 경우

In [10]:
store = {}

In [11]:
response = chain_with_memory.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run f49ebf8b-32c2-41bf-933f-757cf996ff23 not found for run 099e3149-bc1e-4d85-b67a-7875b75395ca. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 172, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9d09112e-b79f-4f48-8f8a-01516b165dd0-0')

In [12]:
response = chain_with_memory.invoke(
    {"input": "주문 좀 바꿀려고요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 9e05294f-0074-4b82-af83-9f36c1462a06 not found for run 7724e30e-ca74-488d-9293-648dabcdded6. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 194, 'total_tokens': 197}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-08a9bce6-ee6e-4012-9aef-de4955fda7e5-0')

In [13]:
response = chain_with_memory.invoke(
    {"input": "product: 모둠떡 quantity: 2, price: 6,000", 
    "order_id": 2},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 0591b3e8-25e6-41f8-a1dd-8c1dfa2ed625 not found for run 2405c970-7ed4-4a95-af5d-0a401bb23651. Treating as a root run.


AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 225, 'total_tokens': 228}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-914ba83c-74e8-4d0c-9f7a-08c8d4022e1c-0')

# 라우트

In [ ]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough.assign(recent_orders=order_cancel_chain)

In [14]:
def change_cancel_route(info):
    print("="*70)
    print("change_cancel_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["recent_orders"].content:
        return RunnableLambda(cancel_order)
    else:
        return RunnableLambda(fetch_recent_orders)

# 함수 결과도 키에 할당 가능한지 실험

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

demo_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 AI 비서야.
            """
        ),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [24]:
demo_chain = demo_prompt | model

In [25]:
def demo(dict):
    return "성공"


In [29]:
demo_func_chain = demo_chain | demo

In [30]:
store = {}

In [31]:
demo_func_chain.invoke(
    {"input": "안녕", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

'성공'

In [33]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough.assign(products=demo_func_chain)

RunnableAssign(mapper={
  products: ChatPromptTemplate(input_variables=['input', 'order_id'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            너는 AI 비서야.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002180020F010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021800230790>, openai_api_key=SecretStr('**********'), openai_proxy='')
            | RunnableLambda(demo)
})

In [34]:
store = {}

In [36]:
RunnablePassthrough.invoke(
    {"input": "안녕", 
    "order_id": None},
    )

TypeError: RunnablePassthrough.invoke() missing 1 required positional argument: 'input'

In [37]:
RunnablePassthrough.invoke()

TypeError: RunnablePassthrough.invoke() missing 2 required positional arguments: 'self' and 'input'

In [38]:
RunnablePassthrough.assign(quried_products=demo_func_chain)

RunnableAssign(mapper={
  quried_products: ChatPromptTemplate(input_variables=['input', 'order_id'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            너는 AI 비서야.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
                   | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002180020F010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021800230790>, openai_api_key=SecretStr('**********'), openai_proxy='')
                   | RunnableLambda(demo)
})

In [39]:
store = {}

In [41]:
RunnablePassthrough.invoke(
    {"input": "안녕", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

TypeError: RunnablePassthrough.invoke() missing 1 required positional argument: 'input'

In [ ]:
store = []

In [43]:
response = chain_with_memory.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run c05054bb-1464-47d4-8b50-8845aea53d31 not found for run 49051faf-2eef-4e77-bc71-dc837cd64403. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 172, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dafb7e33-6285-4853-ad3a-7b55030a3493-0')

In [42]:
query_chain = RunnablePassthrough.assign(quried_products=demo_func_chain)

In [44]:
demo_query_chain = chain_with_memory | query_chain
demo_query_chain

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to review the latest conversation first and output either '주문 변경' or '주문 취소'.\n            If order_id is not provide

In [45]:
response = demo_query_chain.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 9acdae7d-78a8-4772-9f61-a4c1416d91ef not found for run afc44b10-7ae7-4bac-9be7-8b0b2c3bc150. Treating as a root run.


AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [46]:
assign_chain = {
    "input": RunnablePassthrough(),
    "order_id": RunnablePassthrough()} | RunnablePassthrough.assign(quried_products=demo_func_chain)
assign_chain

{
  input: RunnablePassthrough(),
  order_id: RunnablePassthrough()
}
| RunnableAssign(mapper={
    quried_products: ChatPromptTemplate(input_variables=['input', 'order_id'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            너는 AI 비서야.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
                     | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002180020F010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021800230790>, openai_api_key=SecretStr('**********'), openai_proxy='')
                     | RunnableLambda(demo)
  })

In [47]:
response = assign_chain.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    )
response

{'input': {'input': '판매 상품 좀 알려줘', 'order_id': None},
 'order_id': {'input': '판매 상품 좀 알려줘', 'order_id': None},
 'quried_products': '성공'}

# order_id 식별 경우

In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
chain = prompt | model

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
store

{}

* 위 assign_chain 작업과 관련해 뭔가 꼬인 듯
* response 새로 할당 없이 위에 거 사용되고 있었네 ^^;

In [59]:
chain_with_memory.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run f1d554ff-65c8-4d47-b425-80069bbce82f not found for run 6420d8df-739e-4af0-ba5f-112f2894bb97. Treating as a root run.


{'input': {'input': '판매 상품 좀 알려줘', 'order_id': None},
 'order_id': {'input': '판매 상품 좀 알려줘', 'order_id': None},
 'quried_products': '성공'}

restart 후

In [8]:
response = chain_with_memory.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run c497b16d-4eb2-4cea-a0c3-5b078b502da1 not found for run 3ea5f987-94a4-4f42-b788-3ea7be84f7ae. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 182, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0eba9092-9dbe-4255-8367-3a4c73606ac0-0')

In [9]:
response = chain_with_memory.invoke(
    {"input": "판매 상품 좀 알려줘", 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 7046a95c-22fd-4297-8f67-843cf00f38b4 not found for run ac15afdf-d262-47a7-895c-a5b7977873ed. Treating as a root run.


AIMessage(content='조회', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 208, 'total_tokens': 210}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4cb11ed6-60ac-4a98-8602-7a1c0a26c79e-0')

In [ ]:
def fetch_recent_orders(dict):
    return "조회 결과 예시"

In [ ]:
def change_cancel_route(info):
    print("="*70)
    print("change_cancel_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["recent_orders"].content:
        return RunnableLambda(cancel_order)
    else:
        return RunnableLambda(fetch_recent_orders)